In [7]:
import numpy as np

# 入力値NL、NO、配列R、Cを事前に定義する必要があります
NO = 4   # 他の入力値
R = np.array([0.125, 0.750, 0.140, 0.050])  # 抵抗値
C = np.array([0.0, 0.75, 96.0, 0.0])  # 容量値
NL = len(R)  # レイヤーの数

In [8]:
# 初期化
RW = CW = 0.0
for i in range(NL):
    RW += R[i]
    CW += C[i]

RL = np.sqrt(RW * CW)
RS = 0.0
LY = [1] + [0] * (NO + 1)  # Pythonのリストは0からインデックス
XY = [0.0] * (NO + 1)
RZ = [0.0] * NO
CZ = [0.0] * (NO + 1)
K = 0

while K < 24:
    K += 1
    RC = (RL + RS) / 2
    RD = RC**2
    IC = IL = 0
    XI = 0.0
    RW = CW = 0.0

    while True:
        W = (RW + R[IL] * (1 - XI)) * (CW + C[IL] * (1 - XI))
        if W > RD:
            YE = 1.0
            while True:
                W1 = RW + R[IL] * (YE - XI)
                W2 = CW + C[IL] * (YE - XI)
                W = YE - (W1 * W2 - RD) / (R[IL] * W2 + C[IL] * W1)
                if W >= YE:
                    break
                YE = W
            LY[IC + 1] = IL + 1
            XY[IC + 1] = YE
            if IC < NO - 1:
                IC += 1
                XI = YE
                continue
            else:
                RS = RC
                break
        elif IL < NL - 1:
            RW += R[IL] * (1 - XI)
            CW += C[IL] * (1 - XI)
            XI = 0.0
            IL += 1
        else:
            RL = RC
            break

for I in range(1, NO + 1):
    RW = CW = RC = 0.0
    IL = LY[I - 1] - 1  # Pythonのインデックス調整
    XI = XY[I - 1]
    while IL != LY[I] - 1:
        RD = RW + R[IL] * (1 - XI)
        CD = C[IL] * (1 - XI)
        RC += (RW + RD) * CD / 2.0
        RW = RD
        CW += CD
        IL += 1
        XI = 0.0
        if IL == LY[I] - 1: break
    RD = RW + R[IL] * (XY[I] - XI)
    CD = C[IL] * (XY[I] - XI)
    CZ[I] = (RC + (RW + RD) * CD / 2.0) / RD
    CZ[I - 1] += CW + CD - CZ[I]
    RZ[I - 1] = RD

# 結果の出力
for I in range(1, NO + 1):
    print(f"{I}, {LY[I]}, {XY[I]:.4f}, {CZ[I]:.2f}, {RZ[I - 1]:.3f}")

I = NO
print(f"{I + 1}, {LY[I]}, {XY[I]:.4f}, {CZ[I]:.2f}")


1, 3, 0.2533, 36.41, 0.910
2, 3, 0.5066, 24.31, 0.035
3, 3, 0.7598, 30.58, 0.035
4, 4, 1.0000, 4.64, 0.084
5, 4, 1.0000, 4.64
